In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
batch_size=32
train_datagen = ImageDataGenerator(validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/DatasetSegmented",
    target_size=(224,224),
    subset='training') 

validation_generator = train_datagen.flow_from_directory(
    "/content/drive/My Drive/DatasetSegmented", 
    target_size=(224,224),
    subset='validation') 

Found 320 images belonging to 4 classes.
Found 80 images belonging to 4 classes.


In [3]:
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense

img_rows, img_cols, img_channel = 224, 224, 3

base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(img_rows, img_cols, img_channel))

In [4]:

add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(4, activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:

def init_callbacks():
  from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
  base_path = "/content/drive/My Drive/weights/VGG16S/"

 
  trained_models_path = base_path + 'model_weights'
  model_names = trained_models_path + '.{epoch:04d}--{val_loss:.4f}--{val_accuracy:.4f}.h5'
  model_checkpoint = ModelCheckpoint(model_names, monitor = 'val_accuracy', verbose=1,save_best_only=True)

  callbacks = [model_checkpoint]
  return callbacks

In [6]:

hist = model.fit_generator(steps_per_epoch=train_generator.samples // batch_size, generator=train_generator, validation_data= validation_generator, validation_steps=validation_generator.samples // batch_size,epochs=25,callbacks= init_callbacks())

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/25
10/10 [==============================] - 617s 62s/step - loss: 1.9777 - accuracy: 0.2739 - val_loss: 0.6984 - val_accuracy: 0.3906

Epoch 00001: val_accuracy improved from -inf to 0.39062, saving model to /content/drive/My Drive/weights/VGG16S/model_weights.0001--0.6984--0.3906.h5
Epoch 2/25
10/10 [==============================] - 614s 62s/step - loss: 0.5979 - accuracy: 0.3649 - val_loss: 0.4870 - val_accuracy: 0.5469

Epoch 00002: val_accuracy improved from 0.39062 to 0.54688, saving model to /content/drive/My Drive/weights/VGG16S/model_weights.0002--0.4870--0.5469.h5
Epoch 3/25
10/10 [==============================] - 611s 61s/step - loss: 0.4701 - accuracy: 0.5677 - val_loss: 0.4620 - val_accuracy: 0.6250

Epoch 00003: val_accuracy improved from 0.54688 to 0.62500, saving model to /content/drive/My Drive/weights/VGG16S/model_weights.0003--0.4620--0.6250.h5
Epoch 4/25
10/10 [==============================] - 611s 61s/step - loss: 0.4056 - accuracy: 0.6516 - val_loss: 0.4